# IRM Project (Twitter Sentiment Analysis)

#### Project members
   ###### 1. Anuj Agrawal                    1509113020
   ###### 2. Ayush Singh                    1509113031
   ###### 3. Aakash patel                    1509113002
   ###### 4. Abhilash Pandey             1509113003
   ###### 5. Gautam Genda                 1509113041

## Import all the Dependencies

In [80]:
import pandas as pd
import numpy as np
import tweepy
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import warnings 
warnings.filterwarnings('always')

### connection through twitter API

In [4]:

consumer_key = 'Vq4bB2uFtk8HPnTqeZiLeazKD'
consumer_sectret = 'X82CzwP36l1HR2U5g2sMnsoDayq2AEF0LGUWCheD9bcnm4tfjO'

access_token = '912685818661134336-ol8QdSYGhjF40fpSwFikOdS69P7akXL'
access_token_secret = 'Th2c1FUSULuivBHlsjCmkrXI69bEr3FCTcTMpR9PTCJhq'

auth = tweepy.OAuthHandler(consumer_key, consumer_sectret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Fetch tweets by entering query 

In [27]:

public_tweets = api.search(q='indian elections', count=100)
warnings.filterwarnings('always')

In [28]:
tweet_list = []
for tweet in public_tweets:
    tweet_list.append(tweet.text)
    #print(tweet_list)

In [29]:
len(tweet_list)

100

### mark sentiments of each tweet using [textblob](http://textblob.readthedocs.io/en/dev/)

In [31]:

sentiments = [] 
for tweet in public_tweets:
    analysis = TextBlob(tweet.text)
    if 0 < analysis.sentiment.polarity:
        sentiments.append(1)
    else:
        sentiments.append(0)
        
warnings.filterwarnings('always')

### creating dataset from the fetched tweets

In [32]:

combined_list = list(zip(tweet_list, sentiments))
#print(combined_list)

cols = ['tweet', 'sentiment']

data = pd.DataFrame.from_records(combined_list, columns=cols)

print(data)

                                                tweet  sentiment
0   RT @prashantktm: The @htTweets Monday Intervie...          1
1   RT @vineetjaintimes: My prediction-DEEP fake n...          0
2   RT @vineetjaintimes: My prediction-DEEP fake n...          0
3   RT @vineetjaintimes: My prediction-DEEP fake n...          0
4   RT @prashantktm: The @htTweets Monday Intervie...          1
5   RT @airnewsalerts: Record 20 Indian Americans ...          0
6   RT @vineetjaintimes: My prediction-DEEP fake n...          0
7   RT @airnewsalerts: Record 20 Indian Americans ...          0
8   RT @prashantktm: The @htTweets Monday Intervie...          1
9   RT @vineetjaintimes: My prediction-DEEP fake n...          0
10  RT @vineetjaintimes: My prediction-DEEP fake n...          0
11  RT @vineetjaintimes: My prediction-DEEP fake n...          0
12  Top Fiji elections job for New Zealand woman -...          1
13  @ANI The satement of Election wow now they wan...          1
14  Top Fiji elections jo

In [39]:
data['sentiment'].describe()

count    100.000000
mean       0.200000
std        0.402015
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: sentiment, dtype: float64

In [40]:
X = data['tweet']
y = data['sentiment']

print(X.shape)
print(y.shape)

(100,)
(100,)


### Split the dataset into Train-Test datasets

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(X_train.shape)
print(X_test.shape)

(75,)
(25,)


### Create Document Term Matrix using word Frequency

In [54]:
vect = CountVectorizer(stop_words='english')
X_train_counts = vect.fit_transform(X_train)

print(X_train_counts.shape)
print(type(X_train_counts))
print(X_train_counts)

(75, 191)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 38)	1
  (0, 27)	1
  (0, 185)	1
  (0, 28)	1
  (0, 6)	1
  (0, 67)	1
  (0, 0)	1
  (0, 136)	1
  (0, 3)	1
  (0, 142)	1
  (1, 153)	1
  (1, 61)	1
  (1, 182)	1
  (1, 7)	1
  (1, 25)	1
  (1, 87)	1
  (1, 105)	1
  (1, 42)	1
  (1, 32)	1
  (1, 126)	1
  (1, 173)	1
  (1, 38)	1
  (1, 67)	1
  (1, 142)	1
  (2, 159)	1
  :	:
  (72, 146)	1
  (72, 81)	1
  (72, 63)	1
  (72, 1)	1
  (72, 38)	1
  (72, 142)	1
  (73, 91)	1
  (73, 2)	1
  (73, 181)	1
  (73, 186)	1
  (73, 103)	1
  (73, 75)	1
  (73, 47)	1
  (73, 63)	2
  (73, 38)	1
  (74, 38)	1
  (74, 27)	1
  (74, 185)	1
  (74, 28)	1
  (74, 6)	1
  (74, 67)	1
  (74, 0)	1
  (74, 136)	1
  (74, 3)	1
  (74, 142)	1


In [52]:
print(vect.get_feature_names()[:20])
print(vect.get_feature_names()[-20:])

['20', '2019', '9iyk6uy0ej', 'airnewsalerts', 'alaage', 'allegations', 'americans', 'amp', 'ani', 'anipistaayi', 'antidotdot', 'anubhavpatnaik7', 'apr', 'attend', 'attract', 'bc', 'bet', 'bjp', 'bleedbiue111', 'breaking']
['vdkmwvnnok', 'vg8gya2qzp', 'vineetjaintimes', 'voi', 'voters', 'votes', 'want', 'watershed', 'welcome', 'wellwishers', 'woman', 'worldwide', 'worse', 'wow', 'year', 'zealand', 'अम', 'रत', 'लड', 'सद']


### Create Document term matrix using Tf-idf(Term Frequency- Inverse document frequency ).

In [57]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)
print(type(X_train_tfidf))
print(X_train_tfidf)


(75, 191)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 142)	0.14540645614132222
  (0, 3)	0.378558325033633
  (0, 136)	0.3658297503432734
  (0, 0)	0.3543153539413911
  (0, 67)	0.16900165236900064
  (0, 6)	0.3658297503432734
  (0, 28)	0.3658297503432734
  (0, 185)	0.3658297503432734
  (0, 27)	0.3543153539413911
  (0, 38)	0.143473459338603
  (1, 142)	0.19200850357098406
  (1, 67)	0.22316584306859918
  (1, 38)	0.18945599088793724
  (1, 173)	0.283224797727932
  (1, 126)	0.283224797727932
  (1, 32)	0.283224797727932
  (1, 42)	0.27435981498809664
  (1, 105)	0.283224797727932
  (1, 87)	0.283224797727932
  (1, 25)	0.283224797727932
  (1, 7)	0.283224797727932
  (1, 182)	0.283224797727932
  (1, 61)	0.283224797727932
  (1, 153)	0.283224797727932
  (2, 142)	0.10176245751312442
  :	:
  (72, 146)	0.33268915905862345
  (72, 49)	0.3163893498701205
  (72, 151)	0.37835770392188095
  (72, 130)	0.3526385132218122
  (72, 23)	0.3526385132218122
  (72, 36)	0.3526385132218122
  (73, 38)	0.09705387013491863
  (7

### Fitting Naive Bayes classifier

In [67]:
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Pre-processing the test Data

In [68]:
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

print(X_test_tfidf.shape)
print(type(X_test_tfidf))
print(X_test_tfidf)

(25, 191)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 76)	0.6623334127481719
  (0, 38)	0.1858609981430413
  (0, 18)	0.7257893218604363
  (1, 182)	0.283224797727932
  (1, 173)	0.283224797727932
  (1, 153)	0.283224797727932
  (1, 142)	0.19200850357098406
  (1, 126)	0.283224797727932
  (1, 105)	0.283224797727932
  (1, 87)	0.283224797727932
  (1, 67)	0.22316584306859918
  (1, 61)	0.283224797727932
  (1, 42)	0.27435981498809664
  (1, 38)	0.18945599088793724
  (1, 32)	0.283224797727932
  (1, 25)	0.283224797727932
  (1, 7)	0.283224797727932
  (2, 182)	0.283224797727932
  (2, 173)	0.283224797727932
  (2, 153)	0.283224797727932
  (2, 142)	0.19200850357098406
  (2, 126)	0.283224797727932
  (2, 105)	0.283224797727932
  (2, 87)	0.283224797727932
  (2, 67)	0.22316584306859918
  :	:
  (23, 142)	0.09154935472954602
  (23, 121)	0.28305291699891677
  (23, 117)	0.28305291699891677
  (23, 107)	0.30002588631391647
  (23, 98)	0.30002588631391647
  (23, 90)	0.28305291699891677
  (23, 38)	0.09033232066737737

#### Testing the Test Data

In [72]:
y_pred = clf.predict(X_test_tfidf)

print("Accuracy : ",metrics.accuracy_score(y_test, y_pred))

Accuracy :  0.84


## Optimising the model in two steps
### A. Running the Agorithm using a pipeline that includes the following steps
   ##### 1. creating Document-term matrix using word frequency of each word in document
   ##### 2. creating Document matrix using Tf-idf term weighs
   ##### 3. Fitting classifier



In [78]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

text_clf = text_clf.fit(X_train, y_train)

### B. Optimising the algorithm by tunning the Model Parameters using **"GridSearchCV"** 
##### Tunning of Parameters includes-
###### 1. we can choose whether to use uni-gram or bi-gram words for vocabulary
###### 2. we can choose whether to Enable inverse-document-frequency reweighting or not (Desable by default).
###### 3. adjusting the smoothening parameter (alpha  default_value = 1) in naive bayes classifier

In [79]:
parameters = {'vect__ngram_range' : [(1,1), (1,2)],
             'tfidf__use_idf': (True, False),
             'clf__alpha' : (1e-2, 1e-3)
             }

In [81]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (0.01, 0.001)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [83]:
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

### Fitting Model using optimal Tuned parameters

In [87]:
vect1 = CountVectorizer(stop_words='english', ngram_range=(1,2))
X_train_counts1 = vect1.fit_transform(X_train, y_train)

tfidf_transformer1 = TfidfTransformer(use_idf=False)
X_train_tfidf1 = tfidf_transformer1.fit_transform(X_train_counts1)

print(X_train_tfidf1.shape)
print(type(X_train_tfidf1))
print(X_train_tfidf1)

(75, 420)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 58)	0.22941573387056174
  (0, 409)	0.22941573387056174
  (0, 60)	0.22941573387056174
  (0, 15)	0.22941573387056174
  (0, 167)	0.22941573387056174
  (0, 2)	0.22941573387056174
  (0, 301)	0.22941573387056174
  (0, 9)	0.22941573387056174
  (0, 313)	0.22941573387056174
  (0, 82)	0.22941573387056174
  (0, 56)	0.22941573387056174
  (0, 408)	0.22941573387056174
  (0, 59)	0.22941573387056174
  (0, 14)	0.22941573387056174
  (0, 166)	0.22941573387056174
  (0, 0)	0.22941573387056174
  (0, 300)	0.22941573387056174
  (0, 8)	0.22941573387056174
  (0, 312)	0.22941573387056174
  (1, 141)	0.19245008972987526
  (1, 403)	0.19245008972987526
  (1, 17)	0.19245008972987526
  (1, 83)	0.19245008972987526
  (1, 168)	0.19245008972987526
  (1, 53)	0.19245008972987526
  :	:
  (73, 410)	0.2182178902359924
  (73, 237)	0.2182178902359924
  (73, 183)	0.2182178902359924
  (73, 112)	0.2182178902359924
  (73, 144)	0.4364357804719848
  (73, 82)	0.2182178902359924
  (7

In [88]:
clf1 = MultinomialNB(alpha=0.01)
clf1.fit(X_train_tfidf1, y_train)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [89]:
X_test_counts1 = vect1.transform(X_test)
X_test_tfidf1 = tfidf_transformer1.transform(X_test_counts1)

print(X_test_tfidf1.shape)
print(type(X_test_tfidf1))
print(X_test_tfidf1)

(25, 420)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 37)	0.5
  (0, 82)	0.5
  (0, 185)	0.5
  (0, 186)	0.5
  (1, 16)	0.19245008972987526
  (1, 17)	0.19245008972987526
  (1, 52)	0.19245008972987526
  (1, 53)	0.19245008972987526
  (1, 68)	0.19245008972987526
  (1, 69)	0.19245008972987526
  (1, 82)	0.19245008972987526
  (1, 83)	0.19245008972987526
  (1, 100)	0.19245008972987526
  (1, 102)	0.19245008972987526
  (1, 140)	0.19245008972987526
  (1, 141)	0.19245008972987526
  (1, 166)	0.19245008972987526
  (1, 168)	0.19245008972987526
  (1, 207)	0.19245008972987526
  (1, 208)	0.19245008972987526
  (1, 241)	0.19245008972987526
  (1, 242)	0.19245008972987526
  (1, 280)	0.19245008972987526
  (1, 281)	0.19245008972987526
  (1, 312)	0.19245008972987526
  :	:
  (24, 128)	0.19245008972987526
  (24, 129)	0.19245008972987526
  (24, 159)	0.19245008972987526
  (24, 160)	0.19245008972987526
  (24, 175)	0.19245008972987526
  (24, 176)	0.19245008972987526
  (24, 178)	0.19245008972987526
  (24, 179)	0.1924500

### testing accuracy :

In [91]:
y_pred1 = clf1.predict(X_test_tfidf1)

print("Accuracy :", metrics.accuracy_score(y_test, y_pred1))

Accuracy : 0.84


if we are not getting an improved accuracy after optimal tuning of parameters may be we should need to collect more data.

### Analysing the result Obtained

In [92]:
metrics.confusion_matrix(y_test, y_pred1)

array([[18,  0],
       [ 4,  3]])

In [95]:
# All false negative
print(X_test[y_test > y_pred1])

80    @BeingKaNTRi @BleedbIue111 Karnataka elections...
82    RT @TshumanMC: Mama, even those that pretend t...
56    (1) Watched two really great movies made in In...
73    @Joydas No party has ever got near to 50% in I...
Name: tweet, dtype: object


In [96]:
X[80]

'@BeingKaNTRi @BleedbIue111 Karnataka elections kaadu.. General 😜'

In [97]:
X[82]

'RT @TshumanMC: Mama, even those that pretend to be genuine before elections are here. Once voted into office they reveal their true colours…'

In [98]:
X[56]

'(1) Watched two really great movies made in India called #Bhoothnath. Both great family movies, funny touching stor… https://t.co/d9QEGfjbDW'

In [99]:
X[73]

'@Joydas No party has ever got near to 50% in Indian general elections, not even Rajiv Gandhi in 1985.'